In [104]:
# import pandas

import pandas as pd

In [105]:
# import OS and "helper fucntion" based on user's OS system
import os

def main():

    def generate_path(file_name_str):
        return os.path.join(".", "data", file_name_str)
    
    def generate_output_path(new_file_name_str):
        return os.path.join(".", "output_data", new_file_name_str)
    
    monthly_excentus_source_df = pd.read_excel(generate_path("Site_Redeemer_Issuer_Settlement_3160398.xlsx"), skiprows=1)
    site_list_source_df = pd.read_excel(generate_path("Shell_Site_List.xlsx"))
    pre_vendor_discount_source_df = pd.read_excel(generate_path("March_Vendor_Discount.xlsm"))


In [106]:
# Load Data from Excel Sheets

monthly_excentus_source_df = pd.read_excel("Site_Redeemer_Issuer_Settlement_3160398.xlsx", skiprows=1)
site_list_source_df = pd.read_excel("Shell_Site_List.xlsx")
pre_vendor_discount_source_df = pd.read_excel("March_Vendor_Discount.xlsm")


Anaysis Monthly Excentus: Merging of data provided in the Site List based on Site ID to the Monthly Excentus report. Renaming columns to ensure that both files have the correct column heading to merge on.

In [107]:
# Raname Columns

renamed_site_list = site_list_source_df.rename(columns={"Merchant_Id_1": "Site ID"}, inplace=False)


In [108]:
# Drop specified columns

dropped_site_list = renamed_site_list.drop(columns={"Zip Code"}, inplace=False)

In [109]:
# Convert Site ID to string type

final_site_list = dropped_site_list.copy()  
final_site_list["Site ID"] = dropped_site_list["Site ID"].astype(str)


In [110]:
# Drop last two rows with NAN that only show totals

dropped_nan_excentus = monthly_excentus_source_df.drop(monthly_excentus_source_df.tail(2).index, inplace=False)

In [111]:
# Drop specified columns
columns_to_drop = ["Post Date", "Site Name", "Total Discounted Gallons", "Total Discounts Redeemed", 
"Site Funded", "Redemption Fee", "Issuance Fee", "Flat Fee", "Program ID", "Type"]
dropped_col_excentus = dropped_nan_excentus.drop(columns=columns_to_drop, inplace=False)

In [112]:
# Remove .0 from Site ID
final_monthly_excentus = dropped_col_excentus.copy()
final_monthly_excentus['Site ID'] = dropped_col_excentus['Site ID'].astype(str).apply(lambda x: x[:-2] if x.endswith('.0') else x)

In [128]:
# Process previous_vendor_discount :  Rename Column Names Previous Vendor Funded Discount

renamed_pre_vendor_discount = pre_vendor_discount_source_df.rename(columns={"Site ID": "Customer Name", "Site Name": "Customer #"}, inplace=False)

Analysis Grouping data based on Site ID and calculating the sum for each Customer #

In [120]:
# Group monthly excentus by site ID and sum, convert Site ID to type str

grouped_monthly_excentus = monthly_excentus_source_df.groupby("Site ID").sum().reset_index().astype(str)

In [122]:
# Merge Site List to Grouped Monthly Excentus

merged_site_list_excentus = pd.merge(final_site_list, grouped_monthly_excentus, on= "Site ID", how="outer")

In [123]:
# Sort Grouped Excentus by Customer #
sorted_merged_excentus = merged_site_list_excentus.sort_values(by="Customer #")

Create new excel sheet for the current vendor discount to be used at a later date to calculate total receivable per customer

In [124]:
# Create New DF for Current Vendor Discount
current_vendor_discount = sorted_merged_excentus[["Site ID", "Customer #", "Vendor Funded Discounts"]]

In [125]:
# Export vendor_discount_new to Excel

current_vendor_discount.to_excel("May_Vendor_Funded_Discounts.xlsx", index=False)

In [126]:
# Drop Current Month Vendor Funded Discounts

current_month_payables_excentus = sorted_merged_excentus.drop(columns=["Vendor Funded Discounts"], inplace=False)

In [130]:
# Drop NAN Rows

dropped_nan_previous_vendor_discount = renamed_pre_vendor_discount.dropna()

In [131]:
# Merge current_month_payables_excentus with previous_vendor_discount

merged_excentus_vd = pd.merge(dropped_nan_previous_vendor_discount,sorted_merged_excentus,on="Customer #",how="outer",suffixes=('_new', '_curr'))

In [ ]:
merged_excentus_vd

In [132]:
# Fill in NAN with 0 in Vendor Funded Discounts Column

merged_excentus_vd["Vendor Funded Discounts"] = merged_excentus_vd["Vendor Funded Discounts"].fillna(0)

KeyError: 'Vendor Funded Discounts'

In [ ]:
# Round numbers to two decimal points
columns_to_round = ["Vendor Funded Discounts", "Total Receivable - Daily", "Total Payable - Daily"]
merged_excentus_vd[columns_to_round] = merged_excentus_vd[columns_to_round].round(2)

In [ ]:
# Drop Duplicate Columns

merged_excentus_vd_dropped = merged_excentus_vd.drop("Customer Name_new", axis=1, inplace=False)

In [ ]:
# Rename Columns
merged_excentus_vd_renamed = merged_excentus_vd_dropped.rename(columns={"Customer Name_curr": "Customer Name",  "Total Payable - Daily": "Total Payable"}, inplace=False)

In [ ]:
# Add Vendor Funded Discount to Total Receivable
merged_excentus_vd_renamed["Total Receivable"] = merged_excentus_vd_renamed["Total Receivable - Daily"] + merged_excentus_vd_renamed["Vendor Funded Discounts"]

In [ ]:
# Drop Columns and Create Final Total Sums Sheet
columns_to_drop = ['Vendor Funded Discounts', 'Total Receivable - Daily']
total_sum_excentus = merged_excentus_vd_renamed.drop(columns=columns_to_drop, inplace=False)


In [ ]:
# Reorder Columns for Entry
new_order = ["Site ID", "Customer Name", "Customer #", "Total Receivable", "Total Payable"]
pre_mas_entry_excentus = total_sum_excentus[new_order]
pre_mas_entry_excentus

In [ ]:
# Calculate totals across numerical columns
totals = pre_mas_entry_excentus.select_dtypes(include=[float, int]).sum()
totals

In [ ]:
totals_df = pd.DataFrame(totals).T
totals_df.index = ['Totals']  # Set index label for the totals row

# Concatenate totals_df with pre_mas_entry_excentus
df_totals = pd.concat([pre_mas_entry_excentus, totals_df])

In [ ]:
df_totals.to_excel("May_2024_Excentus.xlsx", index=False)
